In [188]:
!pip install pytrends
!pip install google-api-python-client


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# 인기 지표 칼럼
- 연도별 관중 -> O (TARGET)
- 웹 검색량 -> o
- 연도별 기사 개수 -> o
- 인스타 관련 -> x -> 대체: 트위터
- 유튜브 조회수 -> o
 -> pca
# 구단별 인기 기여도 -> 얘는 2024년 기준으로 위에서 분석한 지표를 기준으로 판단
- **얘는 2024년을 기준으로 할 예정**
-> 회귀: 구체적으로 엘롯기가 상위권에 있었으 ㄹ대, 하위권에 있었을 때 -> 매출이 늘었는지를 분석한다. (가설 픽스를 하는 것이 중요하다)
- 2024 구단별 관중
- 구단별 순위 -> 매출로 본다 / 
- 구단별 검색량
- 구단/연도별 기사 개수
- 유튜브 조회수
- 인스타 -> x

In [189]:
import csv
import pandas as pd
# 셀레니움 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException
import time
# 구글 트랜드 라이브러리
from pytrends.request import TrendReq
# 유튜브 조회수 크롤링 라이브러리
from googleapiclient.discovery import build
import datetime

## 인기 지표 칼럼

In [190]:

def save_partial_results(data_dict, data_col_name, filename="partial.csv"):
    """
    중간 데이터 저장 함수 정의
    """
    partial_data = {
        'year': list(data_dict.keys()),
        data_col_name: list(data_dict.values()),
    }
    # Pandas DataFrame으로 변환 후 저장
    pd.DataFrame(partial_data).to_csv(filename, index=False)
    print(f"부분 데이터가 {filename}에 저장되었습니다!")

def get_article_counts_by_year():
    '''
    연도별로 프로야구 관련 기사 개수를 크롤링한다.
    리턴값: 2010년부터 2024년까지 기사 개수를 년도: 기사 개수 딕셔너리 형태로 리턴
    '''
    total_articles_counts = {year: 0 for year in range(2010, 2025)}
    for year in range(2010, 2025):
        for month in range(1, 13):
            for day in range(1, 32):
                # 날짜 형식 맞추기 (예: 20240101)
                date = f"{year}{month:02}{day:02}"

                # 날짜 유효성 검사
                try:
                    time.strptime(date, "%Y%m%d")  # 유효하지 않은 날짜는 ValueError 발생
                except ValueError:
                    continue

                # 하루 기사 개수 계산
                daily_articles = get_daily_article_count(date)
                total_articles_counts[year] += daily_articles
                print(f"{date} 기사 개수: {daily_articles}개 / 총: {total_articles_counts[year]}개")
        save_partial_results(total_articles_counts, 'article_counts_by_year', f"{year}_partial_y_aud_arti.csv")
    return total_articles_counts

def get_daily_article_count(date):
    '''
    하루의 기사 개수를 리턴합니다.
    리턴값: int
    '''
    total_post_counts = 0
    # Selenium Chromedriver 설정
    chromedriver_path = "/usr/local/bin/chromedriver"  # Chromedriver 경로
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)
    # 네이버 야구 페이지 이동
    base_url = f"https://sports.news.naver.com/kbaseball/news/index?isphoto=N&view=photo&date={date}&type=latest"
    try: 
        driver.get(base_url)
        time.sleep(5)  # 페이지 로딩 대기

        # 전체 페이지 수 가져오기
        try:
            while True:
                total_pages_element = driver.find_element(By.CSS_SELECTOR, "div.paginate")
                total_pages = total_pages_element.text.split()
                print(total_pages)
                # 마지막 페이지 찾기
                if total_pages[-1] == "다음":
                    next_button = driver.find_element(By.CLASS_NAME, "next")
                    next_button.click()
                    time.sleep(2)
                else: # 마지막 페이지 기사 개수 + 30 * (마지막 페이지 개수 - 1)
                    try: 
                        last_page_id = total_pages[-1]
                        last_page_button = driver.find_element(By.CSS_SELECTOR, f"a[data-id='{last_page_id}']")
                        last_page_button.click()
                        time.sleep(2)
                    except Exception as e:
                        print(f"{date}: {e}")
                        
                        
                    ul_selector = "div.news_list2 > ul"  # ul 태그의 선택자
                    li_selector = f"{ul_selector} > li"  # ul 아래 li 선택자
                    li_elements = driver.find_elements(By.CSS_SELECTOR, li_selector)
                    total_post_counts = (int(last_page_id)-1) * 30 + len(li_elements)
                    # print(f"ul 아래 li 개수: {len(li_elements)}개")
                    break
        except Exception as e:
            print(f"오류 발생: {e}")

        # 드라이버 종료
        driver.quit()
        # print(f"{date} 기사 개수: {total_post_counts}")
        return total_post_counts
    except WebDriverException as e:
        print(f"URL 로드 중 오류 발생: {e}")
        return 0

def get_web_search_by_year():
    """
    구글 트랜드를 이용한 연도별 웹 검색량을 크롤링한다.
    리턴값: 2010~2024년 웹 검색량을 {'year': 검색량} 형태의 딕셔너리로 반환.
    """
    pytrends = TrendReq(hl="ko", tz=360)
    keywords = ["야구", "KBO", "야구장", "프로야구"]

    # 요청 딜레이 및 에러 방지
    try:
        pytrends.build_payload(keywords, timeframe="2010-01-01 2024-12-31", geo="KR")
        trends = pytrends.interest_over_time()
    except Exception as e:
        print(f"데이터 요청 실패: {e}")
        return {}

    # 데이터 처리
    if not trends.empty:
        trends = trends.reset_index()  # 날짜(date)를 열로 변환
        trends["year"] = trends["date"].dt.year  # 연도 추출

        # 연도별 합산
        yearly_trends = trends.groupby("year")[["야구", "KBO", "야구장", "프로야구"]].sum()

        # 모든 키워드의 합산 열 추가
        yearly_trends["web_search_by_years"] = yearly_trends.sum(axis=1)

        # 연도별 총 검색량 딕셔너리로 반환
        web_search_counts = yearly_trends["web_search_by_years"].to_dict()
        return web_search_counts
    else:
        print("데이터가 비어 있습니다.")
        return {}

def get_kbo_youtube_all_data_by_year(data):
    '''
    연도별 KBO 유튜브 조회수를 크롤링한다.
    리턴값: ~2024년까지 유튜브 조회수, 좋아요수, 댓글수를 판다스로 리턴
    '''
    data['year'] = pd.to_datetime(data['PublishedAt']).dt.year

    # 연도별로 views, likes, comments의 누적합을 계산
    aft_data = data.groupby('year')[['views', 'likes', 'comments']].sum().reset_index()

    # 새로 만들 데이터프레임의 칼럼 이름 변경
    aft_data.columns = ['year', 'youtube_views', 'youtube_likes', 'youtube_comments']

    # 2010~2014년의 데이터를 0으로 채우기
    all_years = pd.DataFrame({'year': range(2010, 2025)})  # 2010~2024년
    aft_data = pd.merge(all_years, aft_data, on='year', how='left').fillna(0)

    # 데이터타입을 정수로 변경
    aft_data['youtube_views'] = aft_data['youtube_views'].astype(int)
    aft_data['youtube_likes'] = aft_data['youtube_likes'].astype(int)
    aft_data['youtube_comments'] = aft_data['youtube_comments'].astype(int)

    return aft_data

# # 중간 데이터 저장 함수 정의
# def save_partial_results(likes_counts, post_counts, filename="partial_y_a_insta.csv"):
#     """
#     likes_counts와 post_counts 데이터를 지정된 파일에 저장합니다.
#     """
#     partial_data = {
#         'year': list(likes_counts.keys()),
#         'instagram_likes_by_year': list(likes_counts.values()),
#         'instagram_posts_by_year': list(post_counts.values()),
#     }
#     # Pandas DataFrame으로 변환 후 저장
#     pd.DataFrame(partial_data).to_csv(filename, index=False)
#     print(f"부분 데이터가 {filename}에 저장되었습니다!")
    
def get_instagram_data_by_year():
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.chrome.service import Service
    from selenium import webdriver
    import time

    # Chromedriver 경로 설정
    chromedriver_path = "/usr/local/bin/chromedriver"
    service = Service(chromedriver_path)
    dr = webdriver.Chrome(service=service)
    dr.set_window_size(414, 800)
    dr.set_page_load_timeout(300)

    # Instagram 로그인
    dr.get('https://www.instagram.com/')
    time.sleep(2)
    id_box = WebDriverWait(dr, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#loginForm > div > div:nth-child(1) > div > label > input")))
    password_box = dr.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(2) > div > label > input")
    login_button = dr.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(3) > button')
    id_box.send_keys("sksohn01@sookmyung.ac.kr")
    password_box.send_keys("sks0hn01!!")
    login_button.click()
    time.sleep(5)

    # KBO 계정으로 이동
    dr.get("https://www.instagram.com/kbo.official/")
    time.sleep(5)

    # 좋아요 수 및 게시물 수 계산
    likes_counts = {year: 0 for year in range(2010, 2025)}
    post_counts = {year: 0 for year in range(2010, 2025)}

    # 무한 스크롤
    SCROLL_PAUSE_TIME = 2
    last_height = dr.execute_script("return document.body.scrollHeight")
    while True:
        dr.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = dr.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 게시물 처리
    cards = dr.find_elements(By.CLASS_NAME, '_aagw')
    for i in range(len(cards)):
        retries = 0
        while retries < 3:  # 재시도 최대 3회
            try:
                cards = dr.find_elements(By.CLASS_NAME, '_aagw')  # 매번 요소 재참조
                card = cards[i]

                # 스크롤 후 클릭
                dr.execute_script("arguments[0].scrollIntoView(true);", card)
                time.sleep(1)

                # 강제 클릭
                ActionChains(dr).move_to_element(card).click().perform()
                time.sleep(2)

                # 좋아요 데이터 처리
                like_element = WebDriverWait(dr, 30).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.x1s688f.x10wh9bi.x1wdrske.x8viiok.x18hxmgj'))
                )
                likes = like_element.text.replace("좋아요 ", "").replace("개", "").strip()
                if "만" in likes:
                    likes = float(likes[:-1]) * 10000

                # 업로드 연도 가져오기
                time_element = WebDriverWait(dr, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'time.x1p4m5qa')))
                upload_time = int(time_element.get_attribute('datetime')[:4])

                # 데이터 누적
                likes_counts[upload_time] += int(likes)
                post_counts[upload_time] += 1
                break  # 성공적으로 처리했으면 재시도 루프 종료
            except Exception as e:
                retries += 1
                print(f"게시물 {i+1} 클릭 중 오류 발생, 재시도 {retries}/3: {e}")
        else:
            print(f"게시물 {i+1} 처리 실패, 다음 게시물로 넘어갑니다.")

        # 뒤로가기
        dr.back()
        time.sleep(2)

    dr.quit()
    return likes_counts, post_counts


In [191]:
# 인기도 칼럼 수집: 관중수(audience_by_year), 연도별 기사 개수(article_counts_by_year), 크보 유튜브 조회수(kbo_youtube_views_by_year), 인스타그램 해시태그 언급량(instagram_hashtags_by_year), 방송 시청률

### 연도별 기사 개수, 관중수 구하기

In [192]:
# 연도별 기사 개수, 관중수 구하기
################ !!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 실행 금지
##############################################################
year = [
    2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
    2020, 2021, 2022, 2023, 2024
]
audience_by_year = [
    5928626, 6810028, 7156157, 6441945, 6509915, 7360530, 8339577, 8400688, 8073742, 7286008, 
    328317, 1228489, 6076074, 8100326, 10887705
]

article_counts = get_article_counts_by_year()
data = {'year': year,
        'audience_by_year': audience_by_year,
        'article_counts_by_year': list(article_counts.values()),
}
year_data = pd.DataFrame(data)
# year_data.to_csv('y_aud_arti.csv', index=False)

[]
오류 발생: list index out of range
20100101 기사 개수: 0개 / 총: 0개
[]
오류 발생: list index out of range
20100102 기사 개수: 0개 / 총: 0개
['1', '2']
20100103 기사 개수: 36개 / 총: 36개
['1', '2', '3', '4']
20100104 기사 개수: 108개 / 총: 144개
['1', '2', '3', '4', '5', '6']
20100105 기사 개수: 170개 / 총: 314개
['1', '2', '3', '4', '5', '6', '7']
20100106 기사 개수: 190개 / 총: 504개
['1', '2', '3', '4', '5']
20100107 기사 개수: 147개 / 총: 651개
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '다음']
['이전', '11']
20100108: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[data-id='11']"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104e89548 cxxbridge1$str$ptr + 3618920
1   chromedriver                        0x0000000104e81d98 cxxbridge1$str$ptr + 3588280
2   chromedriver                       

KeyboardInterrupt: 

### 웹 검색량 데이터

In [ ]:
# 웹 검색량 데이터
data = pd.read_csv("./y_aud_arti.csv")

web_search_counts = get_web_search_by_year()
data['web_search_count_by_year'] = list(web_search_counts.values())

data = pd.DataFrame(data)
data.to_csv('add_web_search.csv', index=False)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


### 유튜브 조회수, 좋아요수, 댓글수 구하기

In [ ]:
# API key와 YouTube API 버전을 세팅
api_key = "API-KEY"
youtube = build('youtube', 'v3', developerKey=api_key)
print('>>> API key 설정')

# 유튜브 모든 채널에는 'Uploads'라는 기본 채널이 있음
# 해당 채널 ID를 가져온 뒤에, 해당 Uploads의 리스트를 다시 호출해오는 함수

# 유튜브 데이터느 매일 변경됨. 오늘 날짜를 Last_update로 기록하기 위해 Datetime을 사용
today = datetime.datetime.now()
nowDate = today.strftime('%Y-%m-%d')

# 채널id를 가지고 비디오 리스트를 가져오는 함수
def get_channel_videos(channel_id):
    # get Uploads playlist id
    res = youtube.channels().list(id=channel_id, part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    res2 = youtube.channels().list(id=channel_id, part='snippet').execute()
    channel_title = res2['items'][0]['snippet']['title']
    print('>>> 대상 채널명: ' + channel_title)
    videos = []
    next_page_token = None

    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                           part='snippet',
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')

        if next_page_token is None:
            break

    return videos

# 채널의 ID를 입력하면, 그 채널에 속한 비디오를 추출
chan_id = input("채널 아이디: ")
videos = get_channel_videos(chan_id)

print('>>> 대상 채널 ID: '+ chan_id)
print('>>> YouTube에서 해당 채널에 속한 모든 비디오 ID 확인 완료')

# 추출된 비디오 리스트에서 video ID만을 추출하여 list로 만든다.
videoid_list = []
for video in videos:
    id_from_api = video['snippet']['resourceId']['videoId']
    videoid_list.append(id_from_api)

# videoid_list에 ID만 모두 추출하여 저장이 되었다.

# 각 비디오에서 데이터를 추출하여, Dataframe을 만들기 위해 빈 list를 생성한다.
title = []
views = []
likes = []
# dislikes = []
comments = []
upload_date = []
print('>>> 데이터 수집 준비 완료')

# 각 비디오에서 데이터를 가져와서 리스트에 추가한다.
print('>>> 개별 비디오 데이터 수집 시작')
for i in range(len(videoid_list)):
    # for i in range(200):
    request = youtube.videos().list(part='snippet,contentDetails,statistics', id=videoid_list[i])
    response = request.execute()
    print(response['items'][0]['statistics'])

    if response['items'] == []:
        title.append('-')
        views.append('-')
        likes.append('-')
        # dislikes.append('-')
        comments.append('-')

    else:
        # result에서 추출
        tname = response['items'][0]['snippet']['title']
        vc = response['items'][0]['statistics'].get('viewCount', 0)
        lc = response['items'][0]['statistics'].get('likeCount', 0)
        cc = response['items'][0]['statistics'].get('commentCount', 0)
        pA = response['items'][0]['snippet']['publishedAt']

        # append
        title.append(tname)
        views.append(int(vc))
        likes.append(int(lc))
        # dislikes.append(dlc)
        comments.append(int(cc))
        upload_date.append(pA)

print('>>> 개별 비디오 데이터 수집 완료')
print('>>> 비디오 URL 정리')
# YouTube API 응답에는 Video URL이 없음. 이를 생성하기 위해 prefix + Video ID로 리스트를 만든다.
vidurl_prefix = 'https://www.youtube.com/watch?v='
vidurl_list = []

for i in range(len(videoid_list)):
    vidurl = vidurl_prefix + videoid_list[i]
    vidurl_list.append(vidurl)

# Google API의 응답은 UTC를 기준으로 한다. KST로 변환이 필요하며, KST는 UTC+9이다.

original_pubdate = []
for i in range(len(upload_date)):
    originaldate = upload_date[i]
    convertedtime = datetime.datetime.strptime(originaldate, '%Y-%m-%dT%H:%M:%SZ')
    KSTdate = datetime.datetime.strptime(originaldate, '%Y-%m-%dT%H:%M:%SZ') + datetime.timedelta(hours=9)
    KST_converted = KSTdate.strftime('%Y-%m-%d %H:%M')
    original_pubdate.append(KST_converted)


# 위에까지 생성된 모든 리스트를 하나의 데이터프레임으로 옮긴다.
print('>>> 데이터프레임 형태로 가공')
sum_df = pd.DataFrame([title, original_pubdate, videoid_list, vidurl_list, views, likes, comments]).T

# 편의를 위해 컬럼 이름을 추가해준다.
sum_df.columns = ['title', 'PublishedAt', 'ID', 'URL', 'views', 'likes', 'comments']

# 유튜브 조회수는 매일 다르므로, 오늘 작업 날짜를 데이터프레임으로 추가한다. 시간은 무시한다.
# 데이터 프레임에 넣기 전에, 비디오 개수만큼 날짜가 들어간 리스트를 만든다.
date_list = []
for i in range(len(videoid_list)):
    date_list.append(nowDate)


# 데이터프레임에 'Last_update_Date'을 추가한다.
print('>>> 오늘 날짜(작업일) 기록 중')
sum_df['Last_updated_Date'] = date_list

# 채널명을 다시 가져온다.
res2 = youtube.channels().list(id=chan_id, part='snippet').execute()
channel_title = res2['items'][0]['snippet']['title']

# 오늘 날짜가 들어간 csv 파일을 생성한다.
print('>>> 작업이 완료되었습니다.')
filename = channel_title + '_' + today.strftime('%Y%m%d') + '.csv'
sum_df.to_csv(filename, encoding='utf-8-sig', index=False)
print('결과물: ',filename)
# CHANNEL_ID = "UCoVz66yWHzVsXAFG8WhJK9g"  # KBO 공식 채널 ID



>>> API key 설정
>>> 대상 채널명: KBO
>>> 대상 채널 ID: UCoVz66yWHzVsXAFG8WhJK9g
>>> YouTube에서 해당 채널에 속한 모든 비디오 ID 확인 완료
>>> 데이터 수집 준비 완료
>>> 개별 비디오 데이터 수집 시작
{'viewCount': '43047', 'likeCount': '1460', 'favoriteCount': '0', 'commentCount': '183'}
{'viewCount': '39071', 'likeCount': '1135', 'favoriteCount': '0', 'commentCount': '133'}
{'viewCount': '3599', 'likeCount': '127', 'favoriteCount': '0', 'commentCount': '3'}
{'viewCount': '4552', 'likeCount': '102', 'favoriteCount': '0', 'commentCount': '7'}
{'viewCount': '13941', 'likeCount': '496', 'favoriteCount': '0', 'commentCount': '11'}
{'viewCount': '15405', 'likeCount': '528', 'favoriteCount': '0', 'commentCount': '47'}
{'viewCount': '40562', 'likeCount': '1245', 'favoriteCount': '0', 'commentCount': '175'}
{'viewCount': '10358', 'likeCount': '319', 'favoriteCount': '0', 'commentCount': '86'}
{'viewCount': '115321', 'likeCount': '1836', 'favoriteCount': '0', 'commentCount': '251'}
{'viewCount': '4500', 'likeCount': '333', 'favoriteCount': '0', 

In [ ]:
# 유튜브 조회수, 좋아요수, 댓글수 구하기

youtube_bef_prep_data = pd.read_csv("./KBO_20241118.csv")
kbo_youtube_all_data = get_kbo_youtube_all_data_by_year(youtube_bef_prep_data)

data = pd.read_csv('./add_web_search.csv')  
# ['year', 'youtube_views', 'youtube_likes', 'youtube_comments']
data['youtube_views_by_years'] = list(kbo_youtube_all_data['youtube_views'])
data['youtube_likes_by_years'] = list(kbo_youtube_all_data['youtube_likes'])
data['youtube_comments_by_years'] = list(kbo_youtube_all_data['youtube_comments'])
year_data = pd.DataFrame(data)
year_data.to_csv('add_youtube.csv', index=False)

### 데이터 종류

---

## 구단별 인기도 평가

In [207]:
def save_df_data_to_csv(data, filename="team_article_counts.csv"):
    """
    팀별 월별 기사 개수를 CSV 파일로 저장하는 함수.
    
    :param data: 팀별 월별 기사 개수 딕셔너리
    :param filename: 저장할 CSV 파일 이름
    """
    rows = []
    
    for team, months in data.items():
        for month, count in months.items():
            rows.append({"team": team, "month": month, "article_count": count})
    
    # 데이터프레임 생성
    df = pd.DataFrame(rows)
    
    # CSV 저장
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"데이터가 {filename}에 저장되었습니다.")

def get_article_counts_by_team():
    '''
    2024년 기준 구단별로 프로야구 관련 기사 개수를 크롤링한다.
    리턴값: 딕셔너리 형태 { 구단명 : {1: 기사 개수, 2: 기사 개수,,,}}
    '''
    # teams = ["HT", "SS", "LG", "OB", "KT", "SK", "LT", "HH", "NC", "WO"]
    teams = ["HT", "SS", "LG"]
    total_articles_counts_by_team = {team: {month: 0 for month in range(1, 13)} for team in teams}
    print(total_articles_counts_by_team)
    
    for team in teams:
        for month in range(11, 13):
            for day in range(10, 17):
                # 날짜 형식 맞추기 (예: 20240101)
                date = f"2024{month:02}{day:02}"

                # 날짜 유효성 검사
                try:
                    time.strptime(date, "%Y%m%d")  # 유효하지 않은 날짜는 ValueError 발생
                except ValueError:
                    continue

                # 하루 기사 개수 계산
                daily_articles = get_daily_article_count_by_team(team, date)
                total_articles_counts_by_team[team][month] += daily_articles
                print(f"{team} -> {date} 기사 개수: {daily_articles}개 / 총: {total_articles_counts_by_team[team][month]}개")
        save_df_data_to_csv(total_articles_counts_by_team, f"{team}_partial_arti.csv")
    return total_articles_counts_by_team

def get_daily_article_count_by_team(team, date):
    '''
    하루의 기사 개수를 리턴합니다.
    리턴값: int
    '''
    total_post_counts = 0
    # Selenium Chromedriver 설정
    chromedriver_path = "/usr/local/bin/chromedriver"  # Chromedriver 경로
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)
    # 네이버 야구 페이지 이동
    base_url = f"https://sports.news.naver.com/kbaseball/news/index?isphoto=N&type=team&team={team}&view=photo&date={date}"
    try: 
        driver.get(base_url)
        time.sleep(5)  # 페이지 로딩 대기

        # 전체 페이지 수 가져오기
        try:
            while True:
                total_pages_element = driver.find_element(By.CSS_SELECTOR, "div.paginate")
                total_pages = total_pages_element.text.split()
                print("total_pages[-1]: ", total_pages[-1])
                # 마지막 페이지 찾기
                if total_pages[-1] == "다음":
                    next_button = driver.find_element(By.CLASS_NAME, "next")
                    next_button.click()
                    time.sleep(2)
                else: # 마지막 페이지 기사 개수 + 30 * (마지막 페이지 개수 - 1)
                    try: 
                        last_page_id = total_pages[-1]
                        last_page_button = driver.find_element(By.CSS_SELECTOR, f"a[data-id='{last_page_id}']")
                        last_page_button.click()
                        time.sleep(2)
                    except Exception as e:
                        print(f"{date}: {e}")
                        
                        
                    ul_selector = "div.news_list2 > ul"  # ul 태그의 선택자
                    li_selector = f"{ul_selector} > li"  # ul 아래 li 선택자
                    li_elements = driver.find_elements(By.CSS_SELECTOR, li_selector)
                    total_post_counts = (int(last_page_id)-1) * 30 + len(li_elements)
                    # print(f"ul 아래 li 개수: {len(li_elements)}개")
                    break
        except Exception as e: # 이유는 알 수 없으나, paginate가 없음에도 html에는 있는 경우
            print(f"오류 발생 (페이지 수 확인 불가): {e}")
            ul_selector = "div.news_list2 > ul"  # ul 태그의 선택자
            li_selector = f"{ul_selector} > li"  # ul 아래 li 선택자
            li_elements = driver.find_elements(By.CSS_SELECTOR, li_selector)
            total_post_counts = len(li_elements)

        print(f"총 기사 개수: {total_post_counts}")

        # 드라이버 종료
        driver.quit()
        # print(f"{date} 기사 개수: {total_post_counts}")
        return total_post_counts
    except WebDriverException as e:
        print(f"URL 로드 중 오류 발생: {e}")
        return 0

### 구단별 기사 개수

In [208]:
article_counts_by_teams = get_article_counts_by_team()
save_df_data_to_csv(article_counts_by_teams, "by_teams_arti.csv")

{'HT': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}, 'SS': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}, 'LG': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}}
오류 발생 (페이지 수 확인 불가): list index out of range
총 기사 개수: 21
HT -> 20241110 기사 개수: 21개 / 총: 21개
total_pages[-1]:  2
총 기사 개수: 51
HT -> 20241111 기사 개수: 51개 / 총: 72개
오류 발생 (페이지 수 확인 불가): list index out of range
총 기사 개수: 17
HT -> 20241112 기사 개수: 17개 / 총: 89개
오류 발생 (페이지 수 확인 불가): list index out of range
총 기사 개수: 30
HT -> 20241113 기사 개수: 30개 / 총: 119개
total_pages[-1]:  2
총 기사 개수: 52
HT -> 20241114 기사 개수: 52개 / 총: 171개
total_pages[-1]:  2
총 기사 개수: 41
HT -> 20241115 기사 개수: 41개 / 총: 212개
오류 발생 (페이지 수 확인 불가): list index out of range
총 기사 개수: 11
HT -> 20241116 기사 개수: 11개 / 총: 223개
오류 발생 (페이지 수 확인 불가): list index out of range
총 기사 개수: 0
HT -> 20241210 기사 개수: 0개 / 총: 0개
오류 발생 (페이지 수 확인 불가): list index out of range
총 기사 개수: 0
HT -> 20241211 기사 개수: 0개 / 총: 0개

---
# Temp

In [202]:
import pandas as pd

def save_article_data_to_csv(article_data, filename="team_article_counts.csv"):
    """
    팀별 월별 기사 개수를 CSV 파일로 저장하는 함수.
    
    :param article_data: 팀별 월별 기사 개수 딕셔너리
    :param filename: 저장할 CSV 파일 이름
    """
    rows = []
    
    for team, months in article_data.items():
        for month, count in months.items():
            rows.append({"team": team, "month": month, "article_count": count})
    
    # 데이터프레임 생성
    df = pd.DataFrame(rows)
    
    # CSV 저장
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"데이터가 {filename}에 저장되었습니다.")

# 예제 데이터
article_data = {
    'HT': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 223, 12: 0},
    'SS': {1: 10, 2: 15, 3: 20, 4: 25, 5: 30, 6: 35, 7: 40, 8: 45, 9: 50, 10: 55, 11: 60, 12: 65},
}

# CSV 저장 함수 호출
save_article_data_to_csv(article_data, filename="team_article_counts.csv")


데이터가 team_article_counts.csv에 저장되었습니다.
